# **Linear Layer**

## **Matrix Multiplication**

### **Basic**

In [3]:
import torch

In [4]:
x = torch.FloatTensor([[1,2],
                       [3,4],
                       [4,5]])
y = torch.FloatTensor([[1,2],
                       [1,2]])


print(x.size(), y.size())

torch.Size([3, 2]) torch.Size([2, 2])


`matmul()` 함수를 통해 행렬 곱을 수행한다.

In [5]:
z = torch.matmul(x,y)
print(z, z.size())

tensor([[ 3.,  6.],
        [ 7., 14.],
        [ 9., 18.]]) torch.Size([3, 2])


### **Batch Matrix Multiplication**

In [6]:
x = torch.FloatTensor(3,3,2)
y = torch.FloatTensor(3,2,3)

`bmm()` 함수를 통해 행렬 곱이 3번 수행되는 연산을 병렬로 동시에 진행할 수 있다.
마지막 2개의 차원을 행렬 취급하여 병렬로 행렬곱 연산을 수행한다.

In [7]:
z = torch.bmm(x,y)
print(z, z.size())

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]) torch.Size([3, 3, 3])


단, `bmm()` 함수를 적용하기 위해서 마지막 2개의 차원을 제외한 다른 차원의 크기는 동일해야한다.

---

## **Linear Layer**

### **Basic**

- W : 행렬 곱 연산
- b : 브로드캐스팅 덧셈 연산

In [8]:
W = torch.FloatTensor([[1,2], # 입력 3개 -> 출력 2개로 반환하는 가중치 행
                       [3,4],
                       [5,6]])
b = torch.FloatTensor([2,2]) # 출력 2차원 더해지는 편향 벡터

이 텐서들을 파라미터로 삼아 선형 계층 함수를 구성할 수 있다.

In [9]:
def linear(x, W, b):
    y = torch.matmul(x,W) + b

    return y

In [10]:
x = torch.FloatTensor(4,3) # 3개의 요소를 갖는 4개의 샘플(배치크기, input 개수)
y = linear(x, W, b)
print(y.size()) # 최종 출력: 샘플 4개 각각의 2차원 결과

torch.Size([4, 2])


**(배치크기,input_dim)×(input_dim,output_dim)=(배치크기,output_dim)**

---

### **torch.nn.Module 클래스 상속 받기**

pytorch에는 nn(neural networks) 패키지가 있고 내부에는 미리 정의된 많은 신경망들이 있다.
그리고 그 신경망들은 torch.nn.Module이라는 추상 클래스를 상속받아 정의되어 있다.
이 추상 클래스를 상속받아 선형 계층을 구현할 수 있다.

In [11]:
import torch.nn as nn

다음 2개의 메서드를 오버라이드 한다.
- `__init__` : 계층 내부에서 필요한 변수를 미리 선언하고 있으며, 또 다른 nn.Module을 상속받은 인스턴스도 소유할 수 있다.
- `forward()` : 계층을 통과하는데 필요한 계산을 수행한다.

In [12]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2): # 3-dimensional input, 2-dimensional output
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__() # 부모 클래스(nn.Module)의 초기화. nn.Module.__init__(self)와 동일함

        self.W = torch.FloatTensor(input_dim, output_dim)
        self.b = torch.FloatTensor(output_dim)

    def forward(self, x):
            y = torch.matmul(x, self.W) + self.b

            return y

In [13]:
linear = MyLinear(3,2)

x = torch.FloatTensor(5,3) # 3개의 input을 갖는 5개의 샘플 
y = linear(x)

y

tensor([[3.5177e-24, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]])

In [14]:
for p in linear.parameters():
    print(p)

이 방법은 내부에 학습할 수 있는 파라미터가 없는 것으로 인식한다.
더 정확한 방법은 `torch.nn.Parameter` 클래스를 활용하는 것이다.

---

### **nn.Parameter 활용**

In [15]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))

    def forward(self, x):
        y = torch.matmul(x, self.W) + self.b

        return y

In [16]:
linear = MyLinear(3,2)
x = torch.FloatTensor(4,3)
y = linear(x)

for p in linear.parameters():
    print(p)

print(y)

Parameter containing:
tensor([[0., 0.],
        [0., 0.],
        [0., 0.]], requires_grad=True)
Parameter containing:
tensor([0., nan], requires_grad=True)
tensor([[0., nan],
        [0., nan],
        [0., nan],
        [0., nan]], grad_fn=<AddBackward0>)


---

### **nn.Linear 활용**

torch.nn에 미리 정의된 선형 계층을 불러올 수 있다.

`torch.nn.Linear(in_features, out_features, bias=True)`
- `in_features': 입력 벡터의 차원
- `out_features`: 출력 벡터의 차원
- `bias`:True면 편향을 추가

In [17]:
linear = nn.Linear(3,2) # 3차원의 입력 벡터를 받아 2차원의 출력을 반환하는 선형 계층
y = linear(x)
x = torch.FloatTensor(2, 3)
y

tensor([[0.3891, 0.2706],
        [0.3891, 0.2706],
        [0.3891, 0.2706],
        [0.3891, 0.2706]], grad_fn=<AddmmBackward0>)

In [18]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.4863,  0.3822,  0.4906],
        [-0.1499,  0.2326, -0.1646]], requires_grad=True)
Parameter containing:
tensor([0.3891, 0.2706], requires_grad=True)


앞서 `nn.Module`을 상속받아 정의한 `MyLinear` 클래스는 내부의 nn.Module 하위 클래스를 소유할 수 있다. 따라서 아래의 방식도 사용 가능하다.

In [25]:
class MyLinear(nn.Module):
    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        y = self.linear(x)

        return y

In [26]:
linear = MyLinear(3,2) # 3차원의 입력 벡터를 받아 2차원의 출력을 반환하는 선형 계층
x = torch.FloatTensor(7, 3)
y = linear(x)
y

tensor([[ 0.4946, -0.1030],
        [ 0.4946, -0.1030],
        [ 0.4946, -0.1030],
        [ 0.4946, -0.1030],
        [ 0.4946, -0.1030],
        [ 0.4946, -0.1030],
        [ 0.4946, -0.1030]], grad_fn=<AddmmBackward0>)

---

## **GPU 사용하기**

[google colab 실습 노트 바로가기](https://colab.research.google.com/drive/1cQMYdPkMOk4o20jlt2eHDe3F7CqjZSQP?authuser=0#scrollTo=VRWw_v-guAQh)